操作流程
1. 根据AMSA结果（GD and Device mixed file） 和 GD DMC 来生成 GD 按照不同频率生成的excel file， "AMSA.xlxs"  
2. 根据groupby function 生成 AMSA1 文件，将所有的GD amsa 结果 通过不同的site 来分类并取AVE VALUE
3. 根据ACC file 里面的 freq and spl 的setup file,去所有所有ACC file 中的 不同频率点里面的ini file，并提取文件里的所有target value， 并生成新的 “New_ini_Target.xlsx"
4. 根据新生成的"New_ini_Target.xlxs"里分布的频率点，AMSA1中寻找不同频率的sheet， 并写入所有频率的target value，生成新的列
--> 5. 根绝"New_ini_Target.xlsx",搜索ACC file 中，不同频率的所有文件，提取spk vol 文件中的不同site 的vol 值，写入到AMSA1

In [1]:
import os
import glob
import re
import configparser
import pandas as pd
import numpy as np
import xlwings as xl
import xlsxwriter
import openpyxl
import matplotlib.pyplot as plt 
import numpy as np
import math
import shutil
import re

In [2]:

%run A601_Column_List.ipynb


ACC_setup_file_Path = ("C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data/CalibrationSetupFile.csv")

dir_path = os.path.dirname(ACC_setup_file_Path)

print(dir_path)


Old_ACC_Folder = "C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data"
New_ACC_Folder=shutil.copytree(Old_ACC_Folder, Old_ACC_Folder+'_new')
New_ACC_Path = os.path.join(os.path.dirname(Old_ACC_Folder), 'Acoustic_Chambers_Calibration_Data_New')
# print(New_ACC_Path)



          0   1      2      3      4      5      6      7      8      9
0      16.5  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
1      20.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
2      35.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
3      75.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
4      80.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
5     300.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
6     900.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
7    1000.0  94  100.0  106.0  112.0  118.0  124.0  127.0  130.0  132.0
8    1100.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
9    3000.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
10   8000.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
11  10000.0  94    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN
[1000.0, 94.0, 100.0, 106.0, 112.0, 118.0, 124.0, 127.0, 130.0, 

In [3]:



regex = re.compile(".*CalibSpeakersVoltageRMS.*")   # 寻找ACC中Speaker voltage file 
regex1 = re.compile(".*SystemPhase_94dBSPL.*")      # 需要ACC中Phase file

writer1 = pd.ExcelWriter('output_spk.xlsx', engine='xlsxwriter')
writer2 = pd.ExcelWriter('output_Phase.xlsx', engine='xlsxwriter')


for value in column_list:
    search_pattern = os.path.join(New_ACC_Folder, f"*{value}*")
    folders = glob.glob(search_pattern)
    if len(folders) > 0:
        for folder in folders:
            print(f"Found folder for value {value}: {folder}")
         
            files = glob.glob(os.path.join(folder, "*.csv"))
            if len(files) > 0:
                for file in files:
                   
                    match = regex.search(file)
                    if match and 'Real' not in file:
                        
                        print(f"Found file: {file}")

                        data=pd.read_csv(file)
                        sheet_name = str(value)

                        data.to_excel(writer1,sheet_name=sheet_name,index=False) 


                    

                for file in files:
                    
                    match = regex1.search(file)
                    if match:
                        
                        print(f"Found file: {file}")

                        data=pd.read_csv(file)
                        sheet_name = str(value)

                        data.to_excel(writer2,sheet_name=sheet_name,index=False) 
                        

            else:
                print(f"No files found in folder {folder}")
    else:
        print(f"No folders found for value {value}")


writer1.save()
writer2.save()




Found folder for value 94: C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data_new\Data_94dBSPL
Found file: C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data_new\Data_94dBSPL\CalibSpeakersVoltageRMS_94dBSPL.csv
Found file: C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data_new\Data_94dBSPL\SystemPhase_94dBSPL.csv
Found folder for value 100: C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data_new\Data_100dBSPL
Found file: C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data_new\Data_100dBSPL\CalibSpeakersVoltageRMS_100dBSPL.csv
Found folder for value 106: C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data_new\Data_106dBSPL
Found file: C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/Acoustic_Chambers_Calibration_Data_new\Data_106dBSPL\CalibSpeakersVoltageRMS_106dBSPL.csv
Found folder for value 112: C:/Users/wanglinl/Deskto

C:\Users\wanglinl\AppData\Local\Temp\ipykernel_21436\3540250873.py:54: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer1.save()
C:\Users\wanglinl\AppData\Local\Temp\ipykernel_21436\3540250873.py:55: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer2.save()


In [4]:
#Phase compensation and ACC file system phase compensation and saved

folder_path = "C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/ACC_Project/output_Phase.xlsx"
data = pd.read_excel(folder_path, sheet_name=None)
# print(data)

AMSA2_Path = 'C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/ACC_Project/AMSA2.xlsx'
AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)

if '94' in data.keys():

    # print('yes')
    data_94 = data['94'] 
    print(data_94)
    # column = list(data_94.columns.values)  # 将 column 转换为列表?????
    # print(data_94)
    # print(column)
    # print(type(column))

    for sheet_name, sheet_data in AMSA2.items():
        # print(sheet_name)
        if 'Phase' in sheet_name:
            match = re.search(r'\d+', sheet_name)
            if match:
                numeric_part = match.group()
                # print(numeric_part)

                if numeric_part in data_94.columns.values:
                    # print('yes')
                    data_94_column = data_94[numeric_part]
                    # print(data_94_column)

    
                    sheet_data['SystemPhase'] = data_94_column
                    sheet_data['deltaPhase']=sheet_data['PHASE']-sheet_data['Phase_New_Target']
                    sheet_data['New_SystemPhase']=sheet_data['SystemPhase']+sheet_data['deltaPhase']
                    sheet_data[numeric_part]=sheet_data['New_SystemPhase']
                    
           
                    for root, dirs, files in os.walk(New_ACC_Path):
                        for file in files:

                            #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                            if file=='SystemPhase_94dBSPL.csv':
                                Spk94DBSPL_file_path=os.path.join(root,file)
                                Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)
                                # print(Spk94DBSPL_data.columns)
                                # print(sheet_name)

                                if numeric_part in Spk94DBSPL_data.columns:
                                    print('yes')
                                    # Spk94DBSPL_data.loc[:, sheet_name] = sheet_data['vol_ratio']*sheet_data['spkVol']
                                    Spk94DBSPL_data.loc[:, numeric_part] = sheet_data[numeric_part]
                                    Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)
                                else:
                                    continue



with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
    for sheet_name, sheet_data in AMSA2.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)           


    Site/Freq     16.5       20       35       75       80      300      900  \
0           1 -155.427 -159.268 -167.774 -175.022 -175.684  178.015  171.141   
1           2 -154.692 -158.483 -167.329 -174.909 -175.600  177.861  171.142   
2           3 -154.369 -157.983 -166.949 -174.614 -175.317  177.924  170.574   
3           4 -155.824 -159.488 -168.030 -175.351 -176.018  177.640  170.906   
4           5 -155.017 -158.746 -167.496 -175.055 -175.743  177.668  170.349   
5           6 -156.923 -160.086 -168.283 -175.359 -176.033  177.774  170.753   
6           7 -155.252 -158.986 -167.740 -175.168 -175.872  177.651  170.380   
7           8 -154.794 -158.638 -167.483 -175.050 -175.725  177.808  170.802   
8           9 -152.058 -156.224 -165.936 -174.231 -174.987  177.921  170.772   
9          10 -153.916 -157.560 -166.629 -174.391 -175.097  178.306  171.317   
10         11 -154.501 -158.451 -167.400 -174.954 -175.655  177.853  170.617   
11         12 -155.559 -159.257 -167.774

In [5]:
#amplitude compensation and ACC file Speaker voltage / RefMic MIC Amplitude compensation and saved

folder_path_1 = "C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/ACC_Project/output_spk.xlsx"
data_1 = pd.read_excel(folder_path_1, sheet_name=None)

AMSA2_Path = 'C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/ACC_Project/AMSA2.xlsx'
AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)

if '94' in data_1.keys():

    # print('yes')
    data_94_1 = data_1['94'] 
    print(data_94_1)


    for sheet_name, sheet_data in AMSA2.items():

        # print(sheet_name)
        # print(sheet_data)
        if sheet_name in data_94_1.columns.values:

            # print('yes')

        
            # print(data_94.columns.values)

            
         
            data_94_1_column = data_94_1[sheet_name]
        
            
            sheet_data['spkVol'] = data_94_1_column
            sheet_data['deltaSens']=sheet_data['New_Target']-sheet_data['SENS']
            sheet_data['vol_ratio']=10**(sheet_data['deltaSens']/20)
            sheet_data['New_spkVol']=sheet_data['vol_ratio']*sheet_data['spkVol']
            sheet_data[sheet_name]=sheet_data['vol_ratio']*sheet_data['spkVol']

         
            for root, dirs, files in os.walk(New_ACC_Path):
                # print(files)
                for file in files:
                    # print(files)    
                    #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                    if file=='CalibSpeakersVoltageRMS_94dBSPL.csv':
                        # print('yes')
                        Spk94DBSPL_file_path=os.path.join(root,file)
                        Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)
                        print(Spk94DBSPL_data)

                        if sheet_name in Spk94DBSPL_data.columns:
                            # print(sheet_name)
                            # Spk94DBSPL_data.loc[:, sheet_name] = sheet_data['vol_ratio']*sheet_data['spkVol']
                            Spk94DBSPL_data.loc[:, sheet_name] = sheet_data[sheet_name]
                            Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)
                    # else:
                        # print('no')

                    #94dBspl ref mic output - amplitude value updated and saved in new ACC folder
                    if file=='ReferenceMicAmplitude_94dBSPL.csv':
                        Ref_output_file=os.path.join(root,file)
                        Ref_output_data=pd.read_csv(Ref_output_file)

                        # print('yes')
                    
                        if sheet_name in Ref_output_data.columns:

                            Ref_output_data.loc[:,sheet_name]=Ref_output_data.loc[:,sheet_name]+sheet_data['deltaSens']
                            Ref_output_data.to_csv(Ref_output_file,index=False)

        


with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
    for sheet_name, sheet_data in AMSA2.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)
    


    Site/Freq   16.5        20        35        75        80       300  \
0           1  2.215  2.224191  2.113231  2.093664  2.078402  2.041863   
1           2  2.121  2.138903  2.027063  2.003469  1.994417  1.967680   
2           3  2.152  2.178444  2.054799  2.019709  2.013774  1.979306   
3           4  2.152  2.138258  2.030626  2.012268  2.002834  1.983236   
4           5  2.198  2.203052  2.084463  2.064876  2.048887  2.021917   
5           6  2.275  2.248571  2.133673  2.099095  2.099043  2.066247   
6           7  2.198  2.216167  2.100273  2.065473  2.067167  2.040798   
7           8  2.137  2.167415  2.058966  2.040352  2.028232  1.999298   
8           9  2.337  2.320998  2.190493  2.154734  2.149259  2.119267   
9          10  2.521  2.442501  2.308630  2.271700  2.262827  2.216073   
10         11  2.075  2.105649  1.996520  1.977900  1.964967  1.935553   
11         12  2.152  2.152311  2.037871  2.019447  2.004168  1.975366   
12         13  2.215  2.191148  2.0768

In [7]:
# THD Compensation 

AMSA2_Path = 'C:/Users/wanglinl/Desktop/Linlin/2_Code_Temp/ACC_Project/AMSA2.xlsx'
AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)


for sheet_name, sheet_data in AMSA2.items():
    # print(sheet_name)
    if sheet_name=='1000':
        # print(sheet_name)
        data_94_THD = sheet_data['New_spkVol']
        # print(data_94_THD)

for sheet_name, sheet_data in AMSA2.items():

    if 'THD' in sheet_name:
        match = re.search(r'\d+', sheet_name)
        if match:
            numeric_part = int(match.group())
            # print(numeric_part)

            sheet_data['94dBspl'] = data_94_THD
            sheet_data['Delta_SPL'] = numeric_part - 94
            sheet_data['Sens']=10**(sheet_data['Delta_SPL']/20)*sheet_data['94dBspl']
            sheet_data[numeric_part]=sheet_data['Sens']

                
            
            for root, dirs, files in os.walk(New_ACC_Path):
                for file in files:

                    #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                    if file=='CalibSpeakersVoltageRMS_'+str(numeric_part)+'dBSPL.csv' and file!='CalibSpeakersVoltageRMS_94dBSPL.csv' : 
                        # print(file)
                        Spk94DBSPL_file_path=os.path.join(root,file)
                        Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)
                        # print(Spk94DBSPL_data.columns)
                        print(numeric_part)
                        # # print(file)
                        # print(Spk94DBSPL_file_path)
                        print(Spk94DBSPL_data)

                        if '1000' in Spk94DBSPL_data.columns:
                            print('yes')
                            # Spk94DBSPL_data.loc[:, sheet_name] = sheet_data['vol_ratio']*sheet_data['spkVol']
                            Spk94DBSPL_data.loc[:, '1000'] = sheet_data[numeric_part]
                            Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)
                        else:
                            print('no')



with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
    for sheet_name, sheet_data in AMSA2.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)    

100
    Site/Freq  16.5  20  35  75  80  300  900      1000  1100  3000  8000  \
0           1     0   0   0   0   0    0    0  2.551180     0     0     0   
1           2     0   0   0   0   0    0    0  2.510861     0     0     0   
2           3     0   0   0   0   0    0    0  2.521632     0     0     0   
3           4     0   0   0   0   0    0    0  2.559187     0     0     0   
4           5     0   0   0   0   0    0    0  2.560969     0     0     0   
5           6     0   0   0   0   0    0    0  2.550984     0     0     0   
6           7     0   0   0   0   0    0    0  2.551340     0     0     0   
7           8     0   0   0   0   0    0    0  2.479859     0     0     0   
8           9     0   0   0   0   0    0    0  2.702976     0     0     0   
9          10     0   0   0   0   0    0    0  2.720375     0     0     0   
10         11     0   0   0   0   0    0    0  2.421424     0     0     0   
11         12     0   0   0   0   0    0    0  2.495682     0     0     